In [1]:
import pandas as pd
from io import StringIO

output = StringIO()
pd.read_csv('./data/raw_data/area/v_region_2023.csv')[['REG', 'LIBELLE', 'CHEFLIEU']].rename(
    columns={'REG': 'code', 'LIBELLE': 'nom', 'CHEFLIEU': 'cheflieu'}).to_csv(output, index=False)

output.seek(0)
output.read()

"code,nom,cheflieu\r\n1,Guadeloupe,97105\r\n2,Martinique,97209\r\n3,Guyane,97302\r\n4,La Réunion,97411\r\n6,Mayotte,97608\r\n11,Île-de-France,75056\r\n24,Centre-Val de Loire,45234\r\n27,Bourgogne-Franche-Comté,21231\r\n28,Normandie,76540\r\n32,Hauts-de-France,59350\r\n44,Grand Est,67482\r\n52,Pays de la Loire,44109\r\n53,Bretagne,35238\r\n75,Nouvelle-Aquitaine,33063\r\n76,Occitanie,31555\r\n84,Auvergne-Rhône-Alpes,69123\r\n93,Provence-Alpes-Côte d'Azur,13055\r\n94,Corse,2A004\r\n"

In [2]:
import tomllib

with open('./data/config.toml', 'rb') as f:
    data = tomllib.load(f)
    print(data)

{'database': {'dbname': 'BDA', 'user': 'vscode', 'password': 'SQL_BDA', 'host': 'localhost', 'port': 5432}, 'region': {'REG': 'code', 'LIBELLE': 'nom'}, 'departement': {'DEP': 'code', 'LIBELLE': 'nom', 'REG': 'code_region'}, 'commune': {'COM': 'code', 'LIBELLE': 'nom', 'DEP': 'code_departement'}, 'cheflieuregion': {'CHEFLIEU': 'code', 'REG': 'code_region'}, 'cheflieudepartement': {'CHEFLIEU': 'code', 'DEP': 'code_departement'}, 'mariage': {'TYPMAR3': 'type_de_mariage', 'REGDEP_MAR': 'region_departement', 'GRAGE': 'groupe_age', 'NBMARIES': 'nombre_de_maries'}, 'premier_mariage': {'TYPMAR3': 'type_de_mariage', 'REGDEP_MAR': 'region_departement', 'GRAGE': 'groupe_age', 'NBMARIES': 'nombre_de_maries'}, 'pays_mariage': {'TYPMAR2': 'type_de_mariage', 'REGDEP_DOMI': 'region_departement', 'LNEPOUX': 'pays_naissance', 'NBMAR': 'nombre_de_maries'}, 'population': {'CODGEO': 'departement_commune', 'SUPERF': 'superficie', 'P20_LOG': 'nombre_logement_2020', 'P14_LOG': 'nombre_logement_2014', 'P09_LO

In [5]:
df = pd.read_csv('./data/raw_data/area/v_commune_2023.csv')
df[df['TYPECOM'] == 'COM']

,TYPECOM,COM,REG,DEP,CTCD,ARR,TNCC,NCC,NCCENR,LIBELLE,CAN,COMPARENT
0,COM,01001,84.0,01,01D,012,5,ABERGEMENT CLEMENCIAT,Abergement-Clémenciat,L'Abergement-Clémenciat,0108,NaN
1,COM,01002,84.0,01,01D,011,5,ABERGEMENT DE VAREY,Abergement-de-Varey,L'Abergement-de-Varey,0101,NaN
2,COM,01004,84.0,01,01D,011,1,AMBERIEU EN BUGEY,Ambérieu-en-Bugey,Ambérieu-en-Bugey,0101,NaN
3,COM,01005,84.0,01,01D,012,1,AMBERIEUX EN DOMBES,Ambérieux-en-Dombes,Ambérieux-en-Dombes,0122,NaN
4,COM,01006,84.0,01,01D,011,1,AMBLEON,Ambléon,Ambléon,0104,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
37558,COM,97613,6.0,976,976D,NaN,0,M TSANGAMOUJI,M'Tsangamouji,M'Tsangamouji,97613,NaN
37559,COM,97614,6.0,976,976D,NaN,1,OUANGANI,Ouangani,Ouangani,97610,NaN
37560,COM,97615,6.0,976,976D,NaN,0,PAMANDZI,Pamandzi,Pamandzi,97611,NaN
37561,COM,97616,6.0,976,976D,NaN,0,SADA,Sada,Sada,97612,NaN


In [4]:
df = pd.read_csv('./data/raw_data/wedding/Dep1.csv')
df

,TYPMAR3;REGDEP_MAR;GRAGE;NBMARIES
0,FF;1175;14_19;0
1,HF-F;1175;14_19;23
2,HF-H;1175;14_19;1
3,HH;1175;14_19;0
4,FF;1177;14_19;0
...,...
4207,HH;FM;TOTAL;5652
4208,FF;FE;TOTAL;7100
4209,HF-F;FE;TOTAL;212413
4210,HF-H;FE;TOTAL;212413
